## PGA Draftkings Notebook
Use [PGA Website](https://www.pgatour.com/tournaments/schedule.html) to look up tournament info and fill out first USER INPUT block below. (Fetch/XHR from Network tab, Paylod sub-tab)

Looks like the tournament ID is also in the address bar.

### User Input

In [1]:
# === USER INPUTS ===
# Old Tournament
old_tournament_name = "RBC Canadian Open"
tournament_date = "6/8/2025"  # Ending date of tournament
old_course = "TPC Toronto at Osprey Valley"
tournament_id = "R2025032"  # Tournament ID from the PGA Tour API - Also visible in the URL of the tournament page

# New Tournament
new_tournament_name = "U.S. Open"
new_ending_date = "6/15/2025"
new_course = "Oakmont Country Club"
new_season = 2025


# === LIBRARIES AND VARIABLES ===
# Import necessary libraries
import requests
import pandas as pd
from datetime import datetime
import sqlite3 as sql
import numpy as np
from numpy import nan
import os
import importlib
import utils.db_utils
from utils.db_utils import TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP

# === TOURNAMENT CONFIG ===
tournament_config = {
    "old": {
        "name": old_tournament_name,
        "date": tournament_date,
        "course": old_course,
        "id": tournament_id
    },
    "new": {
        "name": new_tournament_name,
        "course": new_course,
        "season": new_season,
        "ending_date": pd.to_datetime(new_ending_date, format="%m/%d/%Y"),
        "quoted_course": f'"{new_course}"',
        "quoted_name": f"'{new_tournament_name}'"
    }
}

this_week_key = str(tournament_config["new"]["ending_date"].date())

# Create a minimal one-row history DataFrame
this_week_history = pd.DataFrame([{
    "SEASON": tournament_config["new"]["season"],
    "TOURNAMENT": tournament_config["new"]["name"],
    "ENDING_DATE": tournament_config["new"]["ending_date"],
    "COURSE": tournament_config["new"]["course"],
    "TOURN_ID": None
}])

### Update Database

#### Old Tournament

In [2]:
importlib.reload(utils.db_utils)  # Only needed if you're actively editing db_utils.py
from utils.db_utils import update_tournament_results

# Change these each year!!
season = 2025
year = 20250  # Unique GraphQL year distinguishing number in case of multiple per year

# Run the update
db_path = "data/golf.db"  # Or use os.path.join("data", "golf.db")
tournDf = update_tournament_results(tournament_config, db_path, season, year)

# Show just the most recent tournament added for confirmation
from sqlalchemy import create_engine

engine = create_engine(f"sqlite:///{db_path}")

query = f"""
SELECT *
FROM tournaments
WHERE TOURN_ID = '{tournament_config['old']['id']}'
  AND ENDING_DATE = '{datetime.strptime(tournament_config['old']['date'], '%m/%d/%Y').date()}'
"""

recent = pd.read_sql(query, engine)
engine.dispose()
recent.sort_values(by='FINAL_POS').head()

📦 Fetching results for tournament ID R2025032 (RBC Canadian Open), year: 20250
✅ 156 new rows added for 'RBC Canadian Open'


,SEASON,ENDING_DATE,TOURN_ID,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,ROUNDS:1,ROUNDS:2,ROUNDS:3,ROUNDS:4,OFFICIAL_MONEY,FEDEX_CUP_POINTS
123,2025,2025-06-08,R2025032,RBC Canadian Open,TPC Toronto at Osprey Valley,Ryan Fox,1,1,-4,-4,-6,-4,"$1,764,000.00",500.000
126,2025,2025-06-08,R2025032,RBC Canadian Open,TPC Toronto at Osprey Valley,Sam Burns,2,2,-4,-4,-2,-8,"$1,068,200.00",300.000
75,2025,2025-06-08,R2025032,RBC Canadian Open,TPC Toronto at Osprey Valley,Kevin Yu,3,3,-5,-1,-7,-4,"$676,200.00",190.000
22,2025,2025-06-08,R2025032,RBC Canadian Open,TPC Toronto at Osprey Valley,Cameron Young,T4,4,-2,-4,-5,-5,"$441,000.00",122.500
88,2025,2025-06-08,R2025032,RBC Canadian Open,TPC Toronto at Osprey Valley,Matt McCarty,T4,4,-4,-3,-6,-3,"$441,000.00",122.500


#### Stats

In [3]:
importlib.reload(utils.db_utils)
from utils.db_utils import update_season_stats  # <- This line is essential

# Change these each year!!
statsYear = 2025

stats_df = update_season_stats(statsYear, db_path)
stats_df.head()

✅ Overwrote stats for season 2025 with 998 rows.


,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,...,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON
0,A.J. Ewart,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,596,.1970,2025
1,Aaron Baddeley,168.0,-1.022,178.0,-1.292,141.0,-0.265,4.0,0.536,161.0,...,289.6,116.0,57.29%,172.0,60.48%,96.0,60.24%,445,.2958,2025
2,Aaron Cockerill,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,354,.4031,2025
3,Aaron Rai,27.0,0.732,28.0,0.349,32.0,0.403,103.0,-0.019,127.0,...,286.6,1.0,73.43%,19.0,69.21%,129.0,58.27%,33,2.6441,2025
4,Aaron Wilkin,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,806,.1217,2025


#### Odds
Not usually needed for weekly routine.

**Manual Fix! Odds name cleanup (only needed when joins fail)**

Make sure to update the dictionaries in db_utils.py if new names need to be added.

In [30]:
importlib.reload(utils.db_utils)
from utils.db_utils import clean_odds_names, PLAYER_NAME_MAP, TOURNAMENT_NAME_MAP

db_path = "data/golf.db" 
updated_odds = clean_odds_names(db_path, TOURNAMENT_NAME_MAP, PLAYER_NAME_MAP)
updated_odds.head()

ℹ️ No odds rows required name cleanup.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS,TOURNAMENT_ORIG,PLAYER_ORIG


**Historical Odds Updates**

Only run this when loading in entire year odds at the start of each year or if corrections need to be made.  This will load in the entire year into the database and update it using the dictionary in db_utils.py.

In [31]:
importlib.reload(utils.db_utils)
from utils.db_utils import import_historical_odds

oddsYear = "2022-2023"    # URL segment
season = 2023             # PGA Tour season
db_path = "data/golf.db"

odds_df = import_historical_odds(oddsYear, season, db_path)
odds_df.head()

ℹ️ Historical odds for season 2023 already exist — no new rows added.


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
4,2023,Fortinet Championship,2022-09-18,Hideki Matsuyama,18/1,18.0
5,2023,Fortinet Championship,2022-09-18,Max Homa,18/1,18.0
6,2023,Fortinet Championship,2022-09-18,Corey Conners,20/1,20.0
7,2023,Fortinet Championship,2022-09-18,Maverick McNealy,25/1,25.0
8,2023,Fortinet Championship,2022-09-18,Taylor Pendrith,25/1,25.0


**Not normally needed**

The code below is a way to troubleshoot the odds function that is in db_utils.py.  If it does not pull the stats correctly, we can run it outside of the db_utils.py file and explore what might be wrong with it, iterating until it works, and then using that fix to update db_utils.py.  This is not normally needed, but wanted to leave this in place as a reference because this odds website can be tricky.

In [32]:
import pandas as pd
import numpy as np
import requests
import re
from datetime import datetime
from io import StringIO

# === USER INPUT ===
oddsYear = "2020-2021"    # URL segment
season = 2021        # PGA Tour season

url = f"http://golfodds.com/archives-{oddsYear}.html"
response = requests.get(url)
tables = pd.read_html(StringIO(response.text))
# raw_df = tables[5]  # the actual table of interest
# Find the largest 2-column table that contains at least some odds-like strings
raw_df = None
for tbl in tables:
    if tbl.shape[1] == 2 and tbl.shape[0] > 50:  # Rough filter
        sample = tbl.iloc[:, 1].astype(str).str.contains(r"\d+/\d+").sum()
        if sample > 5:
            raw_df = tbl
            break

if raw_df is None:
    raise ValueError("❌ Could not find valid odds table on the page.")

# === STEP 1: Initial clean-up ===
df = raw_df.dropna(how="all").reset_index(drop=True)
df.columns = ["PLAYER", "ODDS"]

# 🔧 Clean up non-breaking spaces and extra whitespace
df["PLAYER"] = (
    df["PLAYER"]
    .astype(str)
    .str.replace("\xa0", " ", regex=False)
    .str.replace(r"\s+", " ", regex=True)
    .str.strip()
)

df.insert(loc=0, column="SEASON", value=season)
df.insert(loc=1, column="TOURNAMENT", value=np.nan)
df.insert(loc=2, column="ENDING_DATE", value=np.nan)

# === STEP 2: Helper function for parsing date strings ===
def parse_ending_date(text):
    import re
    from datetime import datetime

    # Normalize whitespace and symbols
    text = (
        text.replace("\u2013", "-")
            .replace("–", "-")
            .replace("\xa0", " ")
    )
    text = re.sub(r"\bSept(?!ember)\b", "Sep", text)

    # ✅ Fix typo: "Match" → "March" only when used in a date context
    text = re.sub(r"\bMatch(?=\s+\d{1,2}\s*[-–]\s*\d{1,2},\s*\d{4})", "March", text)

    # Pattern 1: "July 30 - August 2, 2015" or "Oct 29 - Nov 1, 2015"
    match = re.search(r"(\w+)\s\d+\s*-\s*(\w+)\s(\d+),\s(\d{4})", text)
    if match:
        month2, day2, year = match.group(2), match.group(3), match.group(4)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month2} {day2}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 2: "November 21-24, 2024"
    match = re.search(r"(\w+)\s\d+-\d+,\s(\d{4})", text)
    if match:
        month, year = match.group(1), match.group(2)
        day = re.search(r"(\d+)-(\d+)", text).group(2)
        for fmt in ["%B %d, %Y", "%b %d, %Y"]:
            try:
                return datetime.strptime(f"{month} {day}, {year}", fmt).date()
            except ValueError:
                continue

    # Pattern 3: "Sunday, October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%A, %B %d, %Y").date()
    except ValueError:
        pass

    # Pattern 4: "October 20, 2019"
    try:
        return datetime.strptime(text.strip(), "%B %d, %Y").date()
    except ValueError:
        pass

    return None

last_tourn_name = None
last_date = None

# === STEP 3: Iterate block by block ===
final_rows = []
i = 0
last_tourn_name = None
last_end_date = None

while i < len(df) - 4:
    player_i = str(df.loc[i, "PLAYER"])
    player_i2 = str(df.loc[i + 2, "PLAYER"])
    player_i3 = str(df.loc[i + 3, "PLAYER"]).lower()

    # Detect start of a new tournament block
    is_header = (
        pd.isna(df.loc[i, "ODDS"]) and
        pd.isna(df.loc[i + 1, "ODDS"]) and (
            re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", player_i2) or
            re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", player_i2)
        )
    )

    if is_header:
        tourn_name = player_i.strip()
        end_date = parse_ending_date(player_i2)

        # Skip cancelled or empty blocks
        if "cancelled" in player_i3:
            print(f"⚠️ Skipping cancelled tournament: {tourn_name} — {end_date}")
            i += 4
            continue

        # Avoid duplicate block processing
        if tourn_name == last_tourn_name and end_date == last_end_date:
            i += 1
            continue

        print(f"📍 Detected: {tourn_name} — Ending: {end_date}")
        last_tourn_name = tourn_name
        last_end_date = end_date
        i += 4  # Skip header lines

        # Collect all player rows until next header block
        while i < len(df) - 2:
            next_i2 = str(df.loc[i + 2, "PLAYER"])
            is_next_header = (
                pd.isna(df.loc[i, "ODDS"]) and
                pd.isna(df.loc[i + 1, "ODDS"]) and (
                    re.search(r"\w+\s\d+\s*[-–]\s*(\w+\s)?\d+,\s\d{4}", next_i2) or
                    re.search(r"(Monday|Tuesday|Wednesday|Thursday|Friday|Saturday|Sunday),?\s+\w+\s\d{1,2},\s\d{4}", next_i2)
                )
            )
            if is_next_header:
                break

            if pd.notna(df.loc[i, "ODDS"]):
                row = df.loc[i].copy()
                row["TOURNAMENT"] = tourn_name
                row["ENDING_DATE"] = end_date
                final_rows.append(row)
            i += 1
    else:
        i += 1

# === STEP 4: Create cleaned DataFrame ===
clean_df = pd.DataFrame(final_rows)

# ✅ Prevent crash if nothing was parsed
if clean_df.empty or "PLAYER" not in clean_df.columns:
    print(f"⚠️ No valid tournament blocks detected for season {season} ({oddsYear})")
    final_df = pd.DataFrame()  # Safe fallback
else:
    # Remove winner tag
    clean_df["PLAYER"] = clean_df["PLAYER"].str.replace(r"\s\*Winner\*", "", regex=True)

    # Clean odds to numeric
    clean_df["VEGAS_ODDS"] = (
        clean_df["ODDS"]
        .str.replace(",", "")
        .str.extract(r"(\d+)/(\d+)")
        .astype(float)
        .apply(lambda x: x[0] / x[1], axis=1)
    )

    # Final output with source index for debugging
    final_df = clean_df[
        ["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER", "ODDS", "VEGAS_ODDS"]
    ].reset_index(drop=True)

    # Drop non-standard team events (e.g., Presidents Cup, Ryder Cup)
    drop_terms = ["Presidents Cup", "Ryder Cup"]
    final_df = final_df[~final_df["TOURNAMENT"].str.contains("|".join(drop_terms), case=False, na=False)]

    display(final_df.head())



📍 Detected: Safeway Open — Ending: 2020-09-13
📍 Detected: US Open — Ending: 2020-09-20
📍 Detected: R & C Championship — Ending: 2020-09-27
📍 Detected: at Big Cedar Lodge - — Ending: 2020-09-22
📍 Detected: Sanderson Farms Champ — Ending: 2020-10-04
📍 Detected: Shriners H for C Open — Ending: 2020-10-11
📍 Detected: The CJ Cup — Ending: 2020-10-18
📍 Detected: ZOZO CHAMPIONSHIP — Ending: 2020-10-25
📍 Detected: Bermuda Championship — Ending: 2020-11-01
📍 Detected: Vivint Houston Open — Ending: 2020-11-08
📍 Detected: The Masters — Ending: 2020-11-15
📍 Detected: The RSM Classic — Ending: 2020-11-22
📍 Detected: Champions for Change — Ending: 2020-11-27
📍 Detected: Mayakoba Golf Classic — Ending: 2020-12-06
📍 Detected: QBE Shootout — Ending: 2020-12-13
📍 Detected: Sentry Tourn of Champions — Ending: 2021-01-10
📍 Detected: Sony Open in Hawaii — Ending: 2021-01-17
📍 Detected: The American Express — Ending: 2021-01-24
📍 Detected: Abu Dhabi HSBC Champ — Ending: 2021-01-24
📍 Detected: Farmers Insura

,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS
0,2021,Safeway Open,2020-09-13,Phil Mickelson,20/1,20.0
1,2021,Safeway Open,2020-09-13,Si Woo Kim,20/1,20.0
2,2021,Safeway Open,2020-09-13,Brendan Steele,20/1,20.0
3,2021,Safeway Open,2020-09-13,Shane Lowry,25/1,25.0
4,2021,Safeway Open,2020-09-13,Sergio Garcia,30/1,30.0


In [33]:
from datetime import datetime, date
# ✅ Check for non-date types in ENDING_DATE
non_dates = final_df[~final_df["ENDING_DATE"].apply(lambda x: isinstance(x, date))]

print(f"🧪 Rows with invalid ENDING_DATE values: {len(non_dates)}")
display(non_dates.head(10))



🧪 Rows with invalid ENDING_DATE values: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


In [34]:
dupes = final_df.duplicated(subset=["SEASON", "TOURNAMENT", "ENDING_DATE", "PLAYER"], keep=False)

print(f"🚨 Duplicate primary keys in final_df: {dupes.sum()}")
display(final_df[dupes].sort_values(by=["SEASON", "TOURNAMENT", "PLAYER"]))

🚨 Duplicate primary keys in final_df: 0


,SEASON,TOURNAMENT,ENDING_DATE,PLAYER,ODDS,VEGAS_ODDS


## Historical Data

### Pull Relevant Seasons
Do a check to see when this course or tournament have been historically played.

In [4]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_combined_history_seasons

# === USER INPUT ===
seasons = list(range(2016, 2025))  # Adjust as needed
db_path = "data/golf.db"

# Pull course and tournament from config
n_course = tournament_config["new"]["course"]
n_tourn = tournament_config["new"]["name"]

# Fetch relevant history
history_df = get_combined_history_seasons(db_path, course=n_course, tournament=n_tourn, allowed_seasons=seasons)
history_df.head(20)


ℹ️ Found 9 relevant tournaments from course or tournament name.


,SEASON,COURSE,TOURN_ID,TOURNAMENT,ENDING_DATE
690,2016,Oakmont CC,026,U.S. Open,2016-06-19
534,2017,Erin Hills,026,U.S. Open,2017-06-18
234,2018,Shinnecock Hills GC,026,U.S. Open,2018-06-17
78,2019,Pebble Beach Golf Links,026,U.S. Open,2019-06-16
390,2020,Winged Foot Golf Club,026,U.S. Open,2020-09-20
0,2021,Torrey Pines,535,U.S. Open,2021-06-20
1001,2022,The Country Club,026,U.S. Open,2022-06-19
1157,2023,The Los Angeles Country Club,R2023026,U.S. Open,2023-06-18
1313,2024,Pinehurst Resort & Country Club,R2024026,U.S. Open,2024-06-16


### Cut Percentage and FedEx Points
Use a rolling-window approach to look at the most recent cut percentage and how many FedEx cup points have been accumulated recently. This will intentionally not match the PGA Tour stats that start over every year, but will have the same amount of data all the time.  We also add a new feature called Form Density which divides the FedEx Cup Points by the Total Events.

In [5]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts = get_cut_and_fedex_history("data/golf.db", history_df, window_months=9)
# cuts["2024-05-12"].head(20)

for end_date, df in cuts.items():
    print(f"\n📆 {end_date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2016-06-19 — U.S. Open (422 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,17,10,378.0,58.8,22.24,0,2016-06-19,U.S. Open
1,Aaron Wise,1,0,0.0,0.0,0.00,0,2016-06-19,U.S. Open
2,Abraham Ancer,10,2,43.0,20.0,4.30,2,2016-06-19,U.S. Open



📆 2017-06-18 — U.S. Open (405 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,15,7,228.29,46.7,15.22,0,2017-06-18,U.S. Open
1,Aaron Wise,6,4,0.00,66.7,0.00,4,2017-06-18,U.S. Open
2,Abraham Ancer,1,1,0.00,100.0,0.00,1,2017-06-18,U.S. Open



📆 2018-06-17 — U.S. Open (410 players)


...

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,A.J. Ewart,1,0,0.00,0.0,0.00,0,2022-06-19,U.S. Open
1,Aaron Baddeley,5,2,56.96,40.0,11.39,0,2022-06-19,U.S. Open
2,Aaron Beverly,1,0,0.00,0.0,0.00,0,2022-06-19,U.S. Open



📆 2023-06-18 — U.S. Open (503 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,15,11,342.286,73.3,22.82,0,2023-06-18,U.S. Open
1,Aaron Beverly,1,1,0.000,100.0,0.00,1,2023-06-18,U.S. Open
2,Aaron Cockerill,1,0,0.000,0.0,0.00,0,2023-06-18,U.S. Open



📆 2024-06-16 — U.S. Open (469 players)


,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,Aaron Baddeley,14,9,181.975,64.3,13.00,0,2024-06-16,U.S. Open
1,Aaron Cockerill,1,0,0.000,0.0,0.00,0,2024-06-16,U.S. Open
2,Aaron Rai,15,12,442.007,80.0,29.47,6,2024-06-16,U.S. Open


### Recent Form

In [6]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish("data/golf.db", history_df, window_months=9)

# Example preview
for date, df in recent_form.items(): 
    print(f"\n📆 {date} — {df['TOURNAMENT'].iloc[0]} ({len(df)} players)")
    display(df.head(3))


📆 2016-06-19 — U.S. Open (422 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Lee McCoy,1,4.0,5.77,2016-06-19,U.S. Open
1,Cody Gribble,1,8.0,11.54,2016-06-19,U.S. Open
2,Jon Rahm,2,12.5,11.38,2016-06-19,U.S. Open



📆 2017-06-18 — U.S. Open (405 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Keith Mitchell,1,11.0,15.87,2017-06-18,U.S. Open
1,Oscar Fraustro,1,13.0,18.76,2017-06-18,U.S. Open
2,Cam Davis,1,15.0,21.64,2017-06-18,U.S. Open



📆 2018-06-17 — U.S. Open (410 players)


...

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Taylor Montgomery,1,11.0,15.87,2022-06-19,U.S. Open
1,Haotong Li,1,12.0,17.31,2022-06-19,U.S. Open
2,Martin Contini,1,16.0,23.08,2022-06-19,U.S. Open



📆 2023-06-18 — U.S. Open (503 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Joohyung Kim,1,1.0,1.44,2023-06-18,U.S. Open
1,Brooks Koepka,2,1.5,1.37,2023-06-18,U.S. Open
2,Scottie Scheffler,16,8.1,2.86,2023-06-18,U.S. Open



📆 2024-06-16 — U.S. Open (469 players)


,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Bryson DeChambeau,2,4.0,3.64,2024-06-16,U.S. Open
1,Scottie Scheffler,14,4.2,1.55,2024-06-16,U.S. Open
2,Dean Burmester,1,12.0,17.31,2024-06-16,U.S. Open


### Course History

In [7]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

# Filter history_df for only the course we're targeting
target_course = tournament_config["new"]["course"]
course_df = history_df[history_df["COURSE"] == target_course]
course_hist = get_course_history("data/golf.db", course_df)

# View example
for date, df in course_hist.items():
    if not df.empty:
        print(f"\n🏌️‍♂️ Course history for {df['TOURNAMENT'].iloc[0]} on {date}")
        display(df.head(3))

## Training Dataset

In [8]:
pd.set_option("display.max_columns", None)   # Show all columns
importlib.reload(utils.db_utils)
from utils.db_utils import build_training_rows
training_df = build_training_rows(
    db_path,
    history_df,
    cuts,
    recent_form,
    course_hist,
)
training_df.head(10)
# training_df.info()
# training_df.isna().sum().sort_values(ascending=False)

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,TOP_20
0,2016,2016-06-19,U.S. Open,Oakmont CC,Aaron Wise,CUT,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,609.0,0.26,500.0,0.0,0.00,0.00,0.0,90.0,129.84,0
1,2016,2016-06-19,U.S. Open,Oakmont CC,Adam Scott,T18,18,1.0,2.062,20.0,0.522,1.0,1.491,83.0,0.049,129.0,-0.168,9.0,22.88,10.0,3.00,8.0,4.00,13.0,4.58,64.0,166.0,13.0,304.6,153.0,55.55,3.0,70.76,74.0,59.5,6.0,7.07,30.0,100.0,971.35,121.42,8.0,24.6,11.20,1
2,2016,2016-06-19,U.S. Open,Oakmont CC,Alex Noren,CUT,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,26.0,3.63,300.0,NaN,NaN,NaN,NaN,NaN,NaN,0
3,2016,2016-06-19,U.S. Open,Oakmont CC,Andres Gonzales,CUT,90,160.0,-0.469,141.0,-0.222,134.0,-0.112,151.0,-0.135,94.0,0.007,91.0,20.01,78.0,3.06,162.0,4.09,80.0,4.67,145.0,224.0,59.0,294.7,165.0,54.55,81.0,66.17,148.0,55.79,270.0,0.66,1000.0,35.3,52.80,3.11,1.0,80.2,27.75,0
4,2016,2016-06-19,U.S. Open,Oakmont CC,Andrew Johnston,T54,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,79.0,1.83,300.0,NaN,NaN,NaN,NaN,NaN,NaN,0
5,2016,2016-06-19,U.S. Open,Oakmont CC,Andrew Landry,T15,15,172.0,-0.694,11.0,0.594,181.0,-0.892,179.0,-0.396,4.0,0.654,160.0,18.48,136.0,3.09,34.0,4.02,169.0,4.76,5.0,85.0,59.0,294.7,26.0,65.55,117.0,64.63,28.0,61.86,482.0,0.37,1000.0,37.5,20.29,2.54,3.0,80.8,36.77,1
6,2016,2016-06-19,U.S. Open,Oakmont CC,Andy Pope,CUT,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
7,2016,2016-06-19,U.S. Open,Oakmont CC,Andy Sullivan,T23,23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,51.0,2.58,300.0,60.0,0.00,0.00,0.0,60.2,33.60,0
8,2016,2016-06-19,U.S. Open,Oakmont CC,Anirban Lahiri,CUT,90,97.0,0.088,95.0,0.025,90.0,0.143,130.0,-0.080,99.0,-0.020,107.0,19.73,136.0,3.09,97.0,4.05,155.0,4.74,154.0,232.0,82.0,291.7,150.0,56.31,161.0,62.88,138.0,56.4,92.0,1.63,250.0,72.7,278.75,25.34,5.0,51.5,20.73,0
9,2016,2016-06-19,U.S. Open,Oakmont CC,Aron Price,CUT,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,499.0,0.36,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,0


In [9]:
# Play with different years to see if I notice anything wrong
training_df[training_df["SEASON"] == 2024].head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,TOP_20
1157,2024,2024-06-16,U.S. Open,Pinehurst Resort & Country Club,Aaron Rai,T19,19,9.0,1.131,36.0,0.303,8.0,0.676,56.0,0.152,83.0,0.054,99.0,22.10%,38.0,3.01,10.0,3.97,44.0,4.54,53.0,153.0,152.0,293.8,1.0,72.02%,7.0,71.58%,33.0,63.18%,21.0,3.2294,200.0,80.0,442.007,29.47,6.0,41.1,14.82,1
1158,2024,2024-06-16,U.S. Open,Pinehurst Resort & Country Club,Adam Hadwin,CUT,90,112.0,-0.036,101.0,0.016,113.0,-0.057,102.0,0.005,82.0,0.058,63.0,23.00%,52.0,3.02,131.0,4.04,116.0,4.61,123.0,198.0,124.0,297.9,74.0,61.57%,152.0,64.30%,151.0,55.81%,56.0,2.0027,150.0,75.0,1029.292,64.33,1.0,43.3,15.28,0
1159,2024,2024-06-16,U.S. Open,Pinehurst Resort & Country Club,Adam Schenk,CUT,90,146.0,-0.421,53.0,0.212,166.0,-0.520,130.0,-0.114,85.0,0.051,132.0,21.24%,152.0,3.09,131.0,4.04,116.0,4.61,102.0,180.0,82.0,302.4,98.0,59.93%,149.0,64.50%,74.0,60.50%,82.0,1.4461,500.0,66.7,568.417,27.07,0.0,50.9,16.47,0
1160,2024,2024-06-16,U.S. Open,Pinehurst Resort & Country Club,Adam Scott,T32,32,38.0,0.540,33.0,0.322,87.0,0.114,67.0,0.104,29.0,0.398,79.0,22.54%,38.0,3.01,57.0,4.00,44.0,4.54,75.0,170.0,49.0,307.1,121.0,58.28%,102.0,66.35%,45.0,62.09%,20.0,3.2584,150.0,85.7,646.065,46.15,2.0,33.4,12.33,0
1161,2024,2024-06-16,U.S. Open,Pinehurst Resort & Country Club,Adam Svensson,T56,56,33.0,0.592,99.0,0.019,28.0,0.380,39.0,0.193,164.0,-0.453,153.0,20.41%,142.0,3.08,85.0,4.01,121.0,4.62,62.0,158.0,118.0,298.3,40.0,64.18%,88.0,67.13%,60.0,61.39%,112.0,1.1671,400.0,78.3,366.278,15.93,9.0,47.3,14.88,0


### Normalization
***Fix Historical Odds***

Many names do not have historical odds but rather are part of the "field." So giving these "NaN" values the average odds (as I would for missing stats) isn't a good approximation of reality – they are typically the "field" because they are not notable and have poor odds individually.

There are also some crazy odds numbers occasionally (1000/1 or 3000/1) that don't happen all that much and are making that tail too long.

This cell assigns odds of 1000/1 for anyone missing and clips anything larger to 1000/1 to normalize the data better and assign the missing odds more appropriately where they belong.

***OWGR Adjustment***

The NaN's for OWGR are similarly bad players.  So we should assign these the worst ranking instead of the mean and clip it at 1000 just in case there are ever outliers.

For the OWGR score, it should match the lowest score in the dataset to assign that person the same as the worst.

***Recent Form Adjustment***

The NaN's for Recent Form mean that this player has not played any tournaments in the lookback period (9 months).  Similar to Odds and OWGR, I want to punish those that don't play often in my model.  Set these to 90 (i.e. like they miss a lot of cuts - because if they aren't playing they aren't good enough to make it to these tournaments very frequently).  The adj_form feature will also need to be updated here with the new data.

***FedEx Cup Point Adjustment***

The NaN's for FedEx Cup Points means there is no data.  Therefore they should be 0.

In [10]:
# === VEGAS ODDS ===
training_df["VEGAS_ODDS"] = training_df["VEGAS_ODDS"].fillna(1000).clip(upper=1000)

# === OWGR and OWGR_RANK ===
owgr_min = training_df["OWGR"].min(skipna=True)
training_df["OWGR"] = training_df["OWGR"].fillna(owgr_min)
training_df["OWGR_RANK"] = training_df["OWGR_RANK"].fillna(1000).astype(float).clip(upper=1000)

# === RECENT FORM and adj_form ===
training_df["RECENT_FORM"] = training_df["RECENT_FORM"].fillna(90)
if "TOTAL_EVENTS_PLAYED" in training_df.columns:
    training_df["adj_form"] = (
        training_df["RECENT_FORM"] / np.log1p(training_df["TOTAL_EVENTS_PLAYED"])
    ).round(2)

# === FEDEX CUP POINTS ===
training_df["FEDEX_CUP_POINTS"] = training_df["FEDEX_CUP_POINTS"].fillna(0)

training_df.head(5)


,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,TOP_20
0,2016,2016-06-19,U.S. Open,Oakmont CC,Aaron Wise,CUT,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,609.0,0.26,500.0,0.0,0.00,0.00,0.0,90.0,129.84,0
1,2016,2016-06-19,U.S. Open,Oakmont CC,Adam Scott,T18,18,1.0,2.062,20.0,0.522,1.0,1.491,83.0,0.049,129.0,-0.168,9.0,22.88,10.0,3.00,8.0,4.00,13.0,4.58,64.0,166.0,13.0,304.6,153.0,55.55,3.0,70.76,74.0,59.5,6.0,7.07,30.0,100.0,971.35,121.42,8.0,24.6,11.20,1
2,2016,2016-06-19,U.S. Open,Oakmont CC,Alex Noren,CUT,90,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,26.0,3.63,300.0,NaN,0.00,NaN,NaN,90.0,NaN,0
3,2016,2016-06-19,U.S. Open,Oakmont CC,Andres Gonzales,CUT,90,160.0,-0.469,141.0,-0.222,134.0,-0.112,151.0,-0.135,94.0,0.007,91.0,20.01,78.0,3.06,162.0,4.09,80.0,4.67,145.0,224.0,59.0,294.7,165.0,54.55,81.0,66.17,148.0,55.79,270.0,0.66,1000.0,35.3,52.80,3.11,1.0,80.2,27.75,0
4,2016,2016-06-19,U.S. Open,Oakmont CC,Andrew Johnston,T54,54,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,None,NaN,None,79.0,1.83,300.0,NaN,0.00,NaN,NaN,90.0,NaN,0


#### Average the NaNs
At this point, the intentional offsetting of NaNs for poor players is over.  I don't want to penalize statistics or course history because you never know where they might land on the spectrum, so for these NaN values, we will take the mean.

In [11]:
# === Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in training_df.columns:
        training_df[col] = (
            training_df[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
            .astype(float)
        )

# Select only the numeric columns
numeric_columns = training_df.select_dtypes(include=['float64', 'int64'])

# Fill NaN values with the mean of each column
numeric_columns = numeric_columns.fillna(numeric_columns.mean())

# Update the original dataframe with the filled numeric columns
training_df.update(numeric_columns)

training_df.head()

,SEASON,ENDING_DATE,TOURNAMENT,COURSE,PLAYER,POS,FINAL_POS,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,TOP_20
0,2016,2016-06-19,U.S. Open,Oakmont CC,Aaron Wise,CUT,90,67.860908,0.430742,76.81552,0.157297,74.360176,0.198589,82.578331,0.07257,85.317716,0.089129,81.556369,21.612372,82.978038,3.048946,72.011713,4.017057,78.674963,4.617262,79.136164,174.920937,81.590044,298.93675,93.330893,60.902079,92.036603,66.532211,86.197657,59.654568,609.0,0.26,500.0,0.000000,0.00,0.000000,0.00000,90.0,129.840000,0
1,2016,2016-06-19,U.S. Open,Oakmont CC,Adam Scott,T18,18,1.000000,2.062000,20.00000,0.522000,1.000000,1.491000,83.000000,0.04900,129.000000,-0.168000,9.000000,22.880000,10.000000,3.000000,8.000000,4.000000,13.000000,4.580000,64.000000,166.000000,13.000000,304.60000,153.000000,55.550000,3.000000,70.760000,74.000000,59.500000,6.0,7.07,30.0,100.000000,971.35,121.420000,8.00000,24.6,11.200000,1
2,2016,2016-06-19,U.S. Open,Oakmont CC,Alex Noren,CUT,90,67.860908,0.430742,76.81552,0.157297,74.360176,0.198589,82.578331,0.07257,85.317716,0.089129,81.556369,21.612372,82.978038,3.048946,72.011713,4.017057,78.674963,4.617262,79.136164,174.920937,81.590044,298.93675,93.330893,60.902079,92.036603,66.532211,86.197657,59.654568,26.0,3.63,300.0,63.657928,0.00,36.402495,2.17019,90.0,30.475021,0
3,2016,2016-06-19,U.S. Open,Oakmont CC,Andres Gonzales,CUT,90,160.000000,-0.469000,141.00000,-0.222000,134.000000,-0.112000,151.000000,-0.13500,94.000000,0.007000,91.000000,20.010000,78.000000,3.060000,162.000000,4.090000,80.000000,4.670000,145.000000,224.000000,59.000000,294.70000,165.000000,54.550000,81.000000,66.170000,148.000000,55.790000,270.0,0.66,1000.0,35.300000,52.80,3.110000,1.00000,80.2,27.750000,0
4,2016,2016-06-19,U.S. Open,Oakmont CC,Andrew Johnston,T54,54,67.860908,0.430742,76.81552,0.157297,74.360176,0.198589,82.578331,0.07257,85.317716,0.089129,81.556369,21.612372,82.978038,3.048946,72.011713,4.017057,78.674963,4.617262,79.136164,174.920937,81.590044,298.93675,93.330893,60.902079,92.036603,66.532211,86.197657,59.654568,79.0,1.83,300.0,63.657928,0.00,36.402495,2.17019,90.0,30.475021,0


In [12]:
# Check for NaN values
assert training_df.isna().sum().sum() == 0, "🚨 Still missing values!"

In [13]:
# Inspect which columns have NaN values
training_df.isna().sum().sort_values(ascending=False).head(10)


SEASON         0
ENDING_DATE    0
TOURNAMENT     0
COURSE         0
PLAYER         0
POS            0
FINAL_POS      0
SGTTG_RANK     0
SGTTG          0
SGOTT_RANK     0
dtype: int64

### Check Features for Outliers and General Health (Histograms)
Now that all the data is cleaned, let's look at the distribution graphs.

In [ ]:
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
import pandas as pd
import numpy as np

# Select numeric columns with >1 unique value, excluding label
df_numeric = training_df.select_dtypes(include=np.number)
df_numeric = df_numeric.loc[:, df_numeric.nunique() > 1]
df_numeric = df_numeric.drop(columns=["TOP_20"], errors="ignore")

# Create subplots
n_cols = 3
n_rows = int(np.ceil(len(df_numeric.columns) / n_cols))
fig = sp.make_subplots(rows=n_rows, cols=n_cols, subplot_titles=df_numeric.columns)

# Add one histogram per feature with smart binning
for i, column in enumerate(df_numeric.columns):
    row = i // n_cols + 1
    col = i % n_cols + 1

    # Compute dynamic bin count
    col_range = df_numeric[column].max() - df_numeric[column].min()
    n_unique = df_numeric[column].nunique()
    nbins = min(50, max(5, int(n_unique / 2))) if col_range < 10 else 50  # tighter for short-range

    fig.add_trace(
        go.Histogram(
            x=df_numeric[column],
            nbinsx=nbins,
            histnorm="density",
            marker=dict(color="#636EFA"),
            opacity=0.8
        ),
        row=row,
        col=col
    )

# Style the figure
fig.update_layout(
    height=300 * n_rows,
    width=1000,
    title_text="Feature Distributions (with Adaptive Binning)",
    template="plotly_dark",
    showlegend=False
)

fig.show()

# Current Week Data

## Import 2025 Field
**Important!**  Save curren't week's `DKSalaries.csv` into Data folder before this step.

In [35]:
importlib.reload(utils.db_utils)
from utils.db_utils import DK_PLAYER_NAME_MAP

# Load DraftKings player list
dk = pd.read_csv("data/DKSalaries.csv", usecols=["Name", "Salary"])

# Normalize DK names to PGA naming convention
dk["Name"] = dk["Name"].replace(DK_PLAYER_NAME_MAP)

# Rename for consistency
dk = dk.rename(columns={"Name": "PLAYER", "Salary": "SALARY"})

dk.head(10)

,PLAYER,SALARY
0,Scottie Scheffler,14400
1,Rory McIlroy,12400
2,Bryson DeChambeau,11000
3,Xander Schauffele,10400
4,Jon Rahm,10200
5,Collin Morikawa,9800
6,Ludvig Aberg,9600
7,Patrick Cantlay,9300
8,Justin Thomas,9000
9,Tommy Fleetwood,8800


## Vegas Odds
Confirm the [Vegas Odds](http://golfodds.com/weekly-odds.html) page is updated with current tournament.

Note: sometimes this site can give errors and not load correctly.  Just have to do it again later.
Another trick to try is change the url try to run it and then put the url back the way it was.  Worked once, maybe random coincidence.

In [36]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_current_week_odds

odds_current = get_current_week_odds(season=2025, tournament_name=new_tournament_name)
odds_current.head(10)


,SEASON,TOURNAMENT,PLAYER,ODDS,VEGAS_ODDS
1,2025,U.S. Open,Scottie Scheffler,11/4,2.75
2,2025,U.S. Open,Rory McIlroy,14/1,14.00
3,2025,U.S. Open,Xander Schauffele,20/1,20.00
4,2025,U.S. Open,Jon Rahm,12/1,12.00
5,2025,U.S. Open,Bryson DeChambeau,7/1,7.00
6,2025,U.S. Open,Ludvig Aberg,25/1,25.00
7,2025,U.S. Open,Collin Morikawa,25/1,25.00
8,2025,U.S. Open,Viktor Hovland,60/1,60.00
9,2025,U.S. Open,Brooks Koepka,80/1,80.00
10,2025,U.S. Open,Justin Thomas,40/1,40.00


## Cut Percentage and FedEx Points

In [37]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_cut_and_fedex_history

cuts_rolling = get_cut_and_fedex_history(db_path, this_week_history, window_months=9)
cuts_current = cuts_rolling[this_week_key].copy()
cuts_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,CUTS_MADE,FEDEX_CUP_POINTS,CUT_PERCENTAGE,form_density,CONSECUTIVE_CUTS,ENDING_DATE,TOURNAMENT
0,A.J. Ewart,1,0,0.000,0.0,0.00,0,2025-06-15,U.S. Open
1,Aaron Baddeley,16,7,55.951,43.8,3.50,0,2025-06-15,U.S. Open
2,Aaron Rai,15,11,577.480,73.3,38.50,0,2025-06-15,U.S. Open
3,Aaron Wise,1,0,0.000,0.0,0.00,0,2025-06-15,U.S. Open
4,Adam Hadwin,17,11,203.569,64.7,11.97,2,2025-06-15,U.S. Open
5,Adam Long,1,0,0.000,0.0,0.00,0,2025-06-15,U.S. Open
6,Adam Schenk,19,8,257.854,42.1,13.57,1,2025-06-15,U.S. Open
7,Adam Scott,11,9,264.810,81.8,24.07,4,2025-06-15,U.S. Open
8,Adam Svensson,20,11,173.158,55.0,8.66,0,2025-06-15,U.S. Open
9,Adrien Dumont de Chassart,8,3,200.500,37.5,25.06,0,2025-06-15,U.S. Open


## Recent Form

In [38]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_recent_avg_finish

recent_form = get_recent_avg_finish(db_path, this_week_history, window_months=9)
recent_form_current = recent_form[this_week_key].copy()

recent_form_current.head(10)

,PLAYER,TOTAL_EVENTS_PLAYED,RECENT_FORM,adj_form,ENDING_DATE,TOURNAMENT
0,Bryson DeChambeau,2,3.5,3.19,2025-06-15,U.S. Open
1,Scottie Scheffler,13,6.9,2.61,2025-06-15,U.S. Open
2,Jon Rahm,2,11.0,10.01,2025-06-15,U.S. Open
3,John Keefer,1,13.0,18.76,2025-06-15,U.S. Open
4,Bubba Watson,1,14.0,20.20,2025-06-15,U.S. Open
5,Ian Gilligan,1,16.0,23.08,2025-06-15,U.S. Open
6,Taisei Shimizu,1,16.0,23.08,2025-06-15,U.S. Open
7,Joaquin Niemann,2,18.5,16.84,2025-06-15,U.S. Open
8,Tommy Fleetwood,12,18.6,7.25,2025-06-15,U.S. Open
9,Rory McIlroy,9,20.4,8.86,2025-06-15,U.S. Open


## Course History

In [39]:
importlib.reload(utils.db_utils)
from utils.db_utils import get_course_history

course_hist = get_course_history(db_path, this_week_history, lookback_years=7)
course_hist_current = course_hist[this_week_key].copy()

course_hist_current.head(10)


""


## Merged Dataframe

In [40]:
importlib.reload(utils.db_utils)
from utils.db_utils import build_test_rows

this_week = build_test_rows(
    db_path=db_path,
    stats_df=stats_df,
    odds_df=odds_current,
    cuts_df=cuts_current,
    recent_form_df=recent_form_current,
    course_hist_df=course_hist_current,
    dk_df=dk,
    season=tournament_config["new"]["season"]
)

this_week.head()

,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,SALARY
0,Aaron Rai,27.0,0.732,28.0,0.349,32.0,0.403,103.0,-0.019,127.0,-0.137,42.0,22.92%,178.0,3.21,7.0,3.96,32.0,4.55,76.0,172,171.0,286.6,1.0,73.43%,19.0,69.21%,129.0,58.27%,33.0,2.6441,2025.0,125.0,73.3,577.480,38.50,0.0,40.1,14.46,6400
1,Adam Schenk,129.0,-0.200,107.0,-0.013,118.0,-0.102,121.0,-0.085,110.0,-0.045,92.0,21.68%,44.0,3.03,155.0,4.07,32.0,4.55,133.0,211,42.0,306.8,169.0,51.75%,165.0,61.65%,90.0,60.42%,152.0,.8818,2025.0,1000.0,42.1,257.854,13.57,1.0,67.5,22.53,5600
2,Adam Scott,68.0,0.237,54.0,0.227,133.0,-0.199,38.0,0.208,103.0,-0.021,76.0,22.08%,123.0,3.09,140.0,4.06,45.0,4.56,133.0,211,85.0,301.8,126.0,56.61%,134.0,63.75%,130.0,58.24%,42.0,2.3231,2025.0,150.0,81.8,264.810,24.07,4.0,43.6,17.55,6600
3,Akshay Bhatia,72.0,0.227,101.0,0.005,21.0,0.494,155.0,-0.272,12.0,0.502,7.0,25.22%,4.0,2.96,42.0,4.00,162.0,4.70,136.0,214,158.0,292.4,56.0,61.71%,55.0,66.56%,159.0,56.15%,30.0,2.6998,2025.0,125.0,80.0,800.240,53.35,2.0,36.6,13.20,6700
4,Alistair Docherty,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,250.0,.5759,2025.0,1000.0,NaN,NaN,NaN,NaN,NaN,NaN,5400


### Dataframe Normalization

Run the same normalization techniques that I ran on the historical data:
- Any NaN Odds go to 1000/1 and clip all values at 1000/1
- OWGR NaN values go to 1000
- Recent form NaN values act like a MC at 90
- Average everything else.

In [49]:
# === 1. Clean percentage stats stored as strings like '62.5%' ===
percent_stats = ["SCRAMBLING", "DRIVING_ACCURACY", "BIRDIES", "GIR"]

for col in percent_stats:
    if col in this_week.columns:
        this_week[col] = (
            this_week[col]
            .astype(str)
            .str.replace('%', '', regex=False)
            .replace(['None', 'nan', 'NaN', '--', 'DNP', ''], np.nan)
        )
        this_week[col] = pd.to_numeric(this_week[col], errors="coerce")

# === 2. Normalize & assign fallback values for key fields ===
this_week["VEGAS_ODDS"] = pd.to_numeric(this_week["VEGAS_ODDS"], errors="coerce").fillna(1000).clip(upper=1000)

if "OWGR" in this_week.columns:
    this_week["OWGR"] = pd.to_numeric(this_week["OWGR"], errors="coerce")
    max_owgr = this_week["OWGR"].dropna().max()
    this_week["OWGR"] = this_week["OWGR"].fillna(max_owgr).clip(upper=1000)

if "OWGR_RANK" in this_week.columns:
    this_week["OWGR_RANK"] = pd.to_numeric(this_week["OWGR_RANK"], errors="coerce").fillna(1000).clip(upper=1000)

this_week["RECENT_FORM"] = pd.to_numeric(this_week["RECENT_FORM"], errors="coerce").fillna(90)
this_week["FEDEX_CUP_POINTS"] = pd.to_numeric(this_week["FEDEX_CUP_POINTS"], errors="coerce").fillna(0)

if "COURSE_HISTORY" in this_week.columns:
    this_week["COURSE_HISTORY"] = pd.to_numeric(this_week["COURSE_HISTORY"], errors="coerce")
    ch_mean = this_week["COURSE_HISTORY"].mean()
    this_week["COURSE_HISTORY"] = this_week["COURSE_HISTORY"].fillna(ch_mean)

# === 3. Fill all remaining NaNs in numeric columns ===
# Recast everything that might look numeric to be sure
for col in this_week.columns:
    if col not in ["PLAYER", "SALARY", "TOURNAMENT", "SEASON"]:
        try:
            this_week[col] = pd.to_numeric(this_week[col], errors="coerce")
        except Exception:
            continue

numeric_cols = this_week.select_dtypes(include=["number"]).columns.tolist()

for col in numeric_cols:
    if this_week[col].isna().any():
        col_mean = this_week[col].mean()
        this_week[col] = this_week[col].fillna(col_mean if not np.isnan(col_mean) else 0)


In [50]:
# === Final Check: No NaNs Should Remain ===
assert this_week.isna().sum().sum() == 0, "🚨 Still missing values in prediction set!"

In [51]:
# === Check for Remaining Missing Values ===
missing_summary = this_week.isna().sum()
missing_summary = missing_summary[missing_summary > 0].sort_values(ascending=False)

print("🧹 Columns still containing NaN values:")
display(missing_summary)

🧹 Columns still containing NaN values:


Series([], dtype: int64)

In [52]:
this_week.head(10)

,PLAYER,SGTTG_RANK,SGTTG,SGOTT_RANK,SGOTT,SGAPR_RANK,SGAPR,SGATG_RANK,SGATG,SGP_RANK,SGP,BIRDIES_RANK,BIRDIES,PAR_3_RANK,PAR_3,PAR_4_RANK,PAR_4,PAR_5_RANK,PAR_5,TOTAL_DRIVING_RANK,TOTAL_DRIVING,DRIVING_DISTANCE_RANK,DRIVING_DISTANCE,DRIVING_ACCURACY_RANK,DRIVING_ACCURACY,GIR_RANK,GIR,SCRAMBLING_RANK,SCRAMBLING,OWGR_RANK,OWGR,SEASON,VEGAS_ODDS,CUT_PERCENTAGE,FEDEX_CUP_POINTS,form_density,CONSECUTIVE_CUTS,RECENT_FORM,adj_form,SALARY
0,Aaron Rai,27.00,0.732000,28.000,0.3490,32.0000,0.403000,103.000,-0.01900,127.000,-0.137000,42.0000,22.920000,178.000,3.210000,7.0,3.960000,32.00,4.55000,76.0000,172.0000,171.0000,286.6000,1.000,73.430,19.0000,69.21000,129.0000,58.270,33.0,2.6441,2025.0,125.0,73.300000,577.480,38.500000,0.000000,40.1,14.460000,6400
1,Adam Schenk,129.00,-0.200000,107.000,-0.0130,118.0000,-0.102000,121.000,-0.08500,110.000,-0.045000,92.0000,21.680000,44.000,3.030000,155.0,4.070000,32.00,4.55000,133.0000,211.0000,42.0000,306.8000,169.000,51.750,165.0000,61.65000,90.0000,60.420,152.0,0.8818,2025.0,1000.0,42.100000,257.854,13.570000,1.000000,67.5,22.530000,5600
2,Adam Scott,68.00,0.237000,54.000,0.2270,133.0000,-0.199000,38.000,0.20800,103.000,-0.021000,76.0000,22.080000,123.000,3.090000,140.0,4.060000,45.00,4.56000,133.0000,211.0000,85.0000,301.8000,126.000,56.610,134.0000,63.75000,130.0000,58.240,42.0,2.3231,2025.0,150.0,81.800000,264.810,24.070000,4.000000,43.6,17.550000,6600
3,Akshay Bhatia,72.00,0.227000,101.000,0.0050,21.0000,0.494000,155.000,-0.27200,12.000,0.502000,7.0000,25.220000,4.000,2.960000,42.0,4.000000,162.00,4.70000,136.0000,214.0000,158.0000,292.4000,56.000,61.710,55.0000,66.56000,159.0000,56.150,30.0,2.6998,2025.0,125.0,80.000000,800.240,53.350000,2.000000,36.6,13.200000,6700
4,Alistair Docherty,71.75,0.294275,81.825,0.0777,76.3625,0.148825,77.975,0.06785,79.625,0.080738,74.2875,22.274875,91.375,3.065125,70.9,4.014625,86.45,4.60325,88.2125,179.0125,95.3875,300.4025,83.625,60.204,90.3125,65.55875,92.0125,60.345,250.0,0.5759,2025.0,1000.0,66.726531,0.000,44.623061,2.357143,90.0,22.965204,5400
5,Alvaro Ortiz,71.75,0.294275,81.825,0.0777,76.3625,0.148825,77.975,0.06785,79.625,0.080738,74.2875,22.274875,91.375,3.065125,70.9,4.014625,86.45,4.60325,88.2125,179.0125,95.3875,300.4025,83.625,60.204,90.3125,65.55875,92.0125,60.345,351.0,0.4061,2025.0,1000.0,0.000000,0.000,0.000000,0.000000,90.0,129.840000,5600
6,Andrea Pavan,71.75,0.294275,81.825,0.0777,76.3625,0.148825,77.975,0.06785,79.625,0.080738,74.2875,22.274875,91.375,3.065125,70.9,4.014625,86.45,4.60325,88.2125,179.0125,95.3875,300.4025,83.625,60.204,90.3125,65.55875,92.0125,60.345,297.0,0.4834,2025.0,1000.0,66.726531,0.000,44.623061,2.357143,90.0,22.965204,5700
7,Andrew Novak,43.00,0.537000,48.000,0.2390,86.0000,0.048000,29.000,0.25100,82.000,0.077000,115.0000,21.010000,159.000,3.130000,103.0,4.030000,45.00,4.56000,138.0000,215.0000,103.0000,299.4000,112.000,57.720,167.0000,61.21000,45.0000,62.760,36.0,2.5563,2025.0,150.0,60.900000,1513.100,65.790000,2.000000,46.7,14.690000,6300
8,Austen Truslow,71.75,0.294275,81.825,0.0777,76.3625,0.148825,77.975,0.06785,79.625,0.080738,74.2875,22.274875,91.375,3.065125,70.9,4.014625,86.45,4.60325,88.2125,179.0125,95.3875,300.4025,83.625,60.204,90.3125,65.55875,92.0125,60.345,997.0,0.0819,2025.0,1000.0,66.726531,0.000,44.623061,2.357143,90.0,22.965204,5200
9,Ben Griffin,33.00,0.706000,58.000,0.2160,42.0000,0.335000,52.000,0.15600,50.000,0.198000,53.0000,22.570000,44.000,3.030000,42.0,4.000000,45.00,4.56000,72.0000,171.0000,70.0000,303.6000,101.000,58.800,30.0000,68.75000,104.0000,59.720,15.0,3.7290,2025.0,70.0,80.000000,1784.496,71.380000,4.000000,37.4,11.480000,7200


## Correlations
**Last 4 Years**

In [53]:
import plotly.express as px

# Get the 4 most recent seasons (or fewer if not available)
available_seasons = sorted(training_df["SEASON"].unique())[-4:]

# Dictionary to store correlation results
correlation_frames = {}

# Calculate correlations for each season
for season in available_seasons:
    df = training_df[training_df["SEASON"] == season].copy()

    # Numeric features only, drop final result
    numeric = df.select_dtypes(include=["float64", "int64"]).drop(columns=["FINAL_POS", "SEASON"], errors="ignore")

    # Skip if TOP_20 is not present
    if "TOP_20" not in numeric.columns:
        continue

    # Correlation with TOP_20
    cor = numeric.corr().abs()["TOP_20"].drop("TOP_20", errors="ignore").sort_values(ascending=False)
    cor_df = cor.reset_index()
    cor_df.columns = ["Feature", "Correlation"]
    correlation_frames[season] = cor_df

# Plot one bar chart per season
for season, cor_df in correlation_frames.items():
    fig = px.bar(
        cor_df,
        x="Feature",
        y="Correlation",
        title=f"🔎 Correlation to TOP_20 — Season {int(season)}",
        height=500,
        width=1000
    )

    fig.update_layout(
        xaxis_title="Feature",
        yaxis_title="Absolute Correlation to TOP_20",
        template="plotly_dark",
        xaxis_tickangle=-45
    )

    fig.show()


**Average of Last 4 Years**

In [54]:
import pandas as pd
import numpy as np
import plotly.express as px

# === Rebuild correlation matrices for the 4 most recent seasons ===
cor = {}
recent_seasons = sorted(training_df["SEASON"].unique())[-4:]

for season in recent_seasons:
    df_season = training_df[training_df["SEASON"] == season].copy()
    numeric_cols = df_season.select_dtypes(include=["float64", "int64"])
    cor[int(season)] = numeric_cols.corr().abs()

# === Combine into one summary DataFrame ===
cor_df = pd.DataFrame()

for season in recent_seasons:
    df = cor[season].reset_index()
    if "TOP_20" not in df.columns:
        continue
    temp = df[["index", "TOP_20"]].copy()
    temp = temp.rename(columns={"TOP_20": str(season)})
    if cor_df.empty:
        cor_df = temp
    else:
        cor_df = cor_df.merge(temp, on="index", how="outer")

# Drop target and metadata rows
cor_df = cor_df[~cor_df["index"].isin(["TOP_20", "FINAL_POS", "SEASON"])]

# Compute average correlation across seasons
cor_df["AVERAGE"] = cor_df.drop(columns=["index"]).mean(axis=1)
cor_df = cor_df.sort_values(by="AVERAGE", ascending=False)

# === Plotly Bar Chart ===
fig = px.bar(
    cor_df,
    x="index",
    y="AVERAGE",
    title="Average Correlation to TOP_20 (Last 4 Seasons)",
    labels={"index": "Feature", "AVERAGE": "Avg Correlation"},
    template="plotly_dark"
)

fig.update_layout(
    xaxis_tickangle=-45,
    height=600,
    width=1000,
    showlegend=False
)

fig.show()


# ML Model

## Predictors

In [55]:
# === Define Features and Target ===
target_col = "TOP_20"

# Columns to exclude from features
exclude = [
    "PLAYER", "TOURNAMENT", "COURSE", "ENDING_DATE", "SEASON", "TOURN_ID",
    "TOP_20", "FINAL_POS"  
]

# Select numeric feature columns
feature_cols = [
    col for col in training_df.columns
    if col not in exclude and training_df[col].dtype in [np.float64, np.int64]
]

print(f"Selected {len(feature_cols)} features:\n", feature_cols)

Selected 37 features:
 ['SGTTG_RANK', 'SGTTG', 'SGOTT_RANK', 'SGOTT', 'SGAPR_RANK', 'SGAPR', 'SGATG_RANK', 'SGATG', 'SGP_RANK', 'SGP', 'BIRDIES_RANK', 'BIRDIES', 'PAR_3_RANK', 'PAR_3', 'PAR_4_RANK', 'PAR_4', 'PAR_5_RANK', 'PAR_5', 'TOTAL_DRIVING_RANK', 'TOTAL_DRIVING', 'DRIVING_DISTANCE_RANK', 'DRIVING_DISTANCE', 'DRIVING_ACCURACY_RANK', 'DRIVING_ACCURACY', 'GIR_RANK', 'GIR', 'SCRAMBLING_RANK', 'SCRAMBLING', 'OWGR_RANK', 'OWGR', 'VEGAS_ODDS', 'CUT_PERCENTAGE', 'FEDEX_CUP_POINTS', 'form_density', 'CONSECUTIVE_CUTS', 'RECENT_FORM', 'adj_form']


### Random Forest
Initial default model before hyperparameter tuning.

This one actually performed really well (ROC AUC above 0.85).  Strangely doing hyperparameter tuning hurt the model (less than 0.7), so I removed that step and we will just use this default model.  This does cross-validation testing on 5 folds to ensure robust train/test splits.

In [56]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# === Standardize features ===
scaler = StandardScaler()
X_rf_std = scaler.fit_transform(training_df[feature_cols])
y_rf = training_df["TOP_20"]

# === Balance the dataset with SMOTE and undersampling ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
pipeline = Pipeline([("o", over), ("u", under)])
X_rf_bal, y_rf_bal = pipeline.fit_resample(X_rf_std, y_rf)

# === Define and evaluate Random Forest ===
rf = RandomForestClassifier(n_estimators=100, random_state=42)
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(rf, X_rf_bal, y_rf_bal, cv=cv, scoring="roc_auc")

print("📊 Random Forest (All Features)")
print("Cross-validation scores:", scores)
print("Average ROC AUC score: {:.2f}".format(scores.mean()))


📊 Random Forest (All Features)
Cross-validation scores: [0.93575299 0.94540174 0.94599053 0.92853787 0.94743128]
Average ROC AUC score: 0.94


Random Forest Training & Feature Importance

Now we train on all the data (no longer cross-validation folds) and extract the feature importance to see how it compares with linear correlations.


In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.calibration import CalibratedClassifierCV
from sklearn.preprocessing import StandardScaler
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import pandas as pd
import plotly.express as px

# === Resample the full dataset ===
over = SMOTE(sampling_strategy=0.5, k_neighbors=3)
under = RandomUnderSampler(sampling_strategy=0.5)
resample_pipeline = Pipeline([("o", over), ("u", under)])
X_all_resampled, y_all_resampled = resample_pipeline.fit_resample(
    training_df[feature_cols], training_df["TOP_20"]
)

# === Standardize ===
scaler = StandardScaler()
X_all_scaled = scaler.fit_transform(X_all_resampled)

# === Train Calibrated Random Forest ===
base_rf = RandomForestClassifier(random_state=42)
final_model = CalibratedClassifierCV(estimator=base_rf, method="sigmoid", cv=5)
final_model.fit(X_all_scaled, y_all_resampled)

# === Aggregate feature importances across all CV folds ===
all_importances = np.array([
    est.estimator.feature_importances_ for est in final_model.calibrated_classifiers_
])
mean_importance = all_importances.mean(axis=0)

importance_df = pd.DataFrame({
    "Feature": feature_cols,
    "Importance": mean_importance
}).sort_values(by="Importance", ascending=False)

# === Plot feature importances ===
fig = px.bar(
    importance_df,
    x="Feature",
    y="Importance",
    title="Random Forest Feature Importances",
    template="plotly_dark"
)
fig.update_layout(xaxis_tickangle=-45)
fig.show()


Use this model on this week's data to predict the best performers.

In [58]:
# === Prepare test data (this week's players) ===

# Select and standardize features using the same scaler as training
X_test = this_week[feature_cols]
X_test_scaled = scaler.transform(X_test)

# === Predict probabilities for Top 20 finish ===
# CalibratedClassifierCV returns well-calibrated probabilities
this_week["PROBABILITY"] = final_model.predict_proba(X_test_scaled)[:, 1]

# === Sort results by highest predicted probability ===
this_week_sorted = this_week.sort_values(by="PROBABILITY", ascending=False).reset_index(drop=True)

# === Define final column order explicitly ===
columns_to_show = [
    "PLAYER", "SALARY", "PROBABILITY",
    "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", "CUT_PERCENTAGE", 
    "OWGR_RANK", "FEDEX_CUP_POINTS",

    # SG stats
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",

    # Scoring/Performance
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",

    # Driving / Ball Striking
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Keep only columns that exist in the dataframe
columns_to_show = [col for col in columns_to_show if col in this_week_sorted.columns]

# Final dataframe for display/export
export_df = this_week_sorted[columns_to_show].copy()

# Preview top 20
export_df.head(20)


,PLAYER,SALARY,PROBABILITY,RECENT_FORM,VEGAS_ODDS,CUT_PERCENTAGE,OWGR_RANK,FEDEX_CUP_POINTS,SGTTG_RANK,SGOTT_RANK,SGAPR_RANK,SGATG_RANK,SGP_RANK,BIRDIES_RANK,PAR_3_RANK,PAR_4_RANK,PAR_5_RANK,TOTAL_DRIVING_RANK,DRIVING_DISTANCE_RANK,DRIVING_ACCURACY_RANK,GIR_RANK,SCRAMBLING_RANK
0,Bryson DeChambeau,11000,0.766116,3.5,7.00,100.0,10.0,0.000,71.75,81.825,76.3625,77.975,79.625,74.2875,91.375,70.9,86.45,88.2125,95.3875,83.625,90.3125,92.0125
1,Shane Lowry,7800,0.740952,28.1,30.00,84.6,12.0,1373.339,2.00,68.000,2.0000,24.000,103.000,68.0000,7.000,17.0,139.00,72.0000,143.0000,28.000,36.0000,24.0000
2,Sepp Straka,7700,0.608681,30.1,40.00,76.5,8.0,2448.408,4.00,41.000,3.0000,133.000,17.000,2.0000,3.000,7.0,8.00,52.0000,148.0000,11.000,1.0000,77.0000
3,Rory McIlroy,12400,0.606831,20.4,14.00,88.9,2.0,2635.786,10.00,2.000,71.0000,56.000,11.000,11.0000,24.000,17.0,8.00,84.0000,3.0000,173.000,77.0000,14.0000
4,Scottie Scheffler,14400,0.605566,6.9,2.75,100.0,1.0,3500.967,1.00,1.000,1.0000,37.000,25.000,3.0000,24.000,1.0,2.00,11.0000,72.0000,30.000,6.0000,2.0000
5,Keegan Bradley,7400,0.604065,26.1,70.00,92.9,18.0,973.717,3.00,27.000,14.0000,20.000,138.000,54.0000,123.000,17.0,79.00,19.0000,68.0000,47.000,60.0000,115.0000
6,Tommy Fleetwood,8800,0.598866,18.6,30.00,100.0,13.0,1323.648,8.00,50.000,12.0000,26.000,43.000,42.0000,171.000,10.0,93.00,68.0000,135.0000,34.000,128.0000,58.0000
7,Collin Morikawa,9800,0.584253,23.4,25.00,100.0,4.0,1292.158,5.00,19.000,6.0000,92.000,51.000,4.0000,2.000,28.0,79.00,52.0000,157.0000,2.000,44.0000,87.0000
8,Russell Henley,7400,0.579052,29.2,70.00,84.6,7.0,1705.884,17.00,118.000,16.0000,11.000,46.000,16.0000,1.000,28.0,93.00,108.0000,176.0000,14.000,10.0000,78.0000
9,Patrick Cantlay,9300,0.535846,23.1,35.00,92.3,16.0,1135.952,11.00,37.000,10.0000,89.000,61.000,14.0000,135.000,5.0,62.00,47.0000,76.0000,80.000,3.0000,131.0000


### Save to CSV

In [60]:
# Ensure required columns for spreadsheet export
if "COURSE_HISTORY" not in export_df.columns:
    export_df["COURSE_HISTORY"] = "-"

# Define columns to round (FedEx + stats)
columns_to_round = [
    "FEDEX_CUP_POINTS",
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

columns_to_round_1 = [
    "COURSE_HISTORY", "CUT_PERCENTAGE"
]

# Round only the selected columns to 0 decimal places
export_df[columns_to_round] = export_df[columns_to_round].round(0).astype("Int64")
# Round only the selected columns to 1 decimal places
export_df[columns_to_round_1] = export_df[columns_to_round_1].round(1)

# Explicit column order for Excel integration
column_order = [
    "PLAYER", "SALARY", "PROBABILITY", "RECENT_FORM", "COURSE_HISTORY", "VEGAS_ODDS", 
    "CUT_PERCENTAGE", "OWGR_RANK", "FEDEX_CUP_POINTS", 
    "SGTTG_RANK", "SGOTT_RANK", "SGAPR_RANK", "SGATG_RANK", "SGP_RANK",
    "BIRDIES_RANK", "PAR_3_RANK", "PAR_4_RANK", "PAR_5_RANK",
    "TOTAL_DRIVING_RANK", "DRIVING_DISTANCE_RANK", "DRIVING_ACCURACY_RANK",
    "GIR_RANK", "SCRAMBLING_RANK"
]

# Ensure all expected columns exist
for col in column_order:
    if col not in export_df.columns:
        export_df[col] = "-" if col == "COURSE_HISTORY" else np.nan

# Reorder columns to match Excel layout
export_df = export_df[column_order]

# Export to CSV
filename = f"data/current_week_export.csv"
export_df.to_csv(filename, index=False)
print(f"✅ Exported to {filename}")

✅ Exported to data/current_week_export.csv
